**Question 1: What is the sum of the outputs of the generator for limit = 5?**



In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value
    print(sqrt_value)

print(f'Sum: {sum}')

**Question 2: What is the 13th number yielded**

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum = sum + sqrt_value
    print(sqrt_value)

print(f'Sum: {sum}')


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989
Sum: 32.854555867161245


**Setup dlt and duckdb**

In [ ]:
!pip install -U dlt
!pip install "dlt[duckdb]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.2 MB/s eta 0:00:00


**Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.**

In [ ]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')

In [ ]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

In [ ]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

In [ ]:
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.39 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707818969.9697757 is LOADED and contains no failed jobs


In [ ]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [ ]:
# and the data

print("\n\n\n people table below:")

peeps = conn.sql("SELECT * FROM people").df()
display(peeps)




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707818969.9697757,fvAKw5I0m/pYQA
1,2,Person_2,27,City_A,1707818969.9697757,Sw7ZPDidWl8bjg
2,3,Person_3,28,City_A,1707818969.9697757,iRY1aXp0QshVBg
3,4,Person_4,29,City_A,1707818969.9697757,2Zgwpmlxo++z0Q
4,5,Person_5,30,City_A,1707818969.9697757,X1iEY0qjFMV2YQ


In [ ]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

In [ ]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

In [ ]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [ ]:
# and the data

print("\n\n\n people table below:")

peeps = conn.sql("SELECT * FROM people").df()
display(peeps)




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707818969.9697757,fvAKw5I0m/pYQA,None
1,2,Person_2,27,City_A,1707818969.9697757,Sw7ZPDidWl8bjg,None
2,3,Person_3,28,City_A,1707818969.9697757,iRY1aXp0QshVBg,None
3,4,Person_4,29,City_A,1707818969.9697757,2Zgwpmlxo++z0Q,None
4,5,Person_5,30,City_A,1707818969.9697757,X1iEY0qjFMV2YQ,None
5,3,Person_3,33,City_B,1707819412.3333519,b3Q39rUKv1TcFA,Job_3
6,4,Person_4,34,City_B,1707819412.3333519,hduXCFzQFrajiw,Job_4
7,5,Person_5,35,City_B,1707819412.3333519,x3vFpJBH9ixNtA,Job_5
8,6,Person_6,36,City_B,1707819412.3333519,EhckZJbxVmz+zg,Job_6
9,7,Person_7,37,City_B,1707819412.3333519,uxuBMrnwtWeX0g,Job_7


In [ ]:
sum_age = conn.sql("SELECT SUM(age) FROM people").df()
display(sum_age)

,sum(age)
0,353.0


**Question 4: Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.**

In [ ]:
# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key="id")

In [ ]:
info = generators_pipeline.run(people_2(),
										table_name="people_merge",
										write_disposition="merge",
                    primary_key="id")

In [ ]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_merge        │
└─────────────────────┘

In [ ]:
# and the data

print("\n\n\n people table below:")

peeps_merge = conn.sql("SELECT * FROM people_merge").df()
display(peeps_merge)




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707819998.032295,cNGOU8D7amHi0g,None
1,1,Person_1,26,City_A,1707819998.032295,X8lfUTG9oRvBbg,None
2,8,Person_8,38,City_B,1707820001.2763011,t0zPyR/tIAoG4Q,Job_8
3,4,Person_4,34,City_B,1707820001.2763011,z4ryP3mIsk4I1w,Job_4
4,5,Person_5,35,City_B,1707820001.2763011,vAOsaR7aHnaXBw,Job_5
5,7,Person_7,37,City_B,1707820001.2763011,J7AIukmXfOSgDQ,Job_7
6,3,Person_3,33,City_B,1707820001.2763011,tObavirv6npV+A,Job_3
7,6,Person_6,36,City_B,1707820001.2763011,97lECEN6z6iwgA,Job_6


In [ ]:
sum_age_merge = conn.sql("SELECT SUM(age) FROM people_merge").df()
display(sum_age_merge)

,sum(age)
0,266.0


In [ ]:
# code from homework starting notebook

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}
